In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cafeteria/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cafeteria/test.csv')

In [62]:
train.count()

일자                1205
요일                1205
본사정원수             1205
본사휴가자수            1205
본사출장자수            1205
본사시간외근무명령서승인건수    1205
현본사소속재택근무자수       1205
조식메뉴              1205
중식메뉴              1205
석식메뉴              1205
중식계               1205
석식계               1205
dtype: int64

In [61]:
test.count()

일자                50
요일                50
본사정원수             50
본사휴가자수            50
본사출장자수            50
본사시간외근무명령서승인건수    50
현본사소속재택근무자수       50
조식메뉴              50
중식메뉴              50
석식메뉴              50
dtype: int64

In [297]:
all_data = pd.concat([train, test], sort=False)
all_data = train

In [298]:
all_data['year'] = pd.to_numeric(all_data['일자'].str[:4])
all_data['month'] = pd.to_numeric(all_data['일자'].str[5:7])
all_data['day'] = pd.to_numeric(all_data['일자'].str[8:10])


weeks = ['월', '화', '수', '목', '금']
for i in range(0, 5):
  all_data.loc[all_data['요일'] == weeks[i], 'wDay'] = i+1

In [299]:
all_data = all_data.drop('일자', axis=1)
all_data = all_data.drop('요일', axis=1)

all_data.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,year,month,day,wDay
0,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,2016,2,1,1.0
1,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,2016,2,2,2.0
2,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,2016,2,3,3.0
3,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,2016,2,4,4.0
4,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,2016,2,5,5.0


In [300]:
all_data['left_members_at_lunch'] = all_data['본사정원수'] - all_data['본사휴가자수'] - all_data['본사출장자수'] - all_data['현본사소속재택근무자수']
all_data['left_members_at_dinner'] = all_data['본사시간외근무명령서승인건수']

In [301]:
all_data = all_data.drop('본사정원수', axis=1)
all_data = all_data.drop('본사휴가자수', axis=1)
all_data = all_data.drop('본사출장자수', axis=1)
all_data = all_data.drop('본사시간외근무명령서승인건수', axis=1)
all_data = all_data.drop('현본사소속재택근무자수', axis=1)

all_data.head()

,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner
0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,2016,2,1,1.0,2401.0,238
1,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,2016,2,2,2.0,2378.0,319
2,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,2016,2,3,3.0,2365.0,111
3,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,2016,2,4,4.0,2277.0,355
4,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,2016,2,5,5.0,2142.0,34


In [288]:
all_data.describe()

,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner
count,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000
mean,890.334440,461.772614,2018.052282,6.512033,15.947718,3.003320,2365.253112,274.117012
std,209.505057,139.179202,1.433958,3.453906,8.650452,1.415384,176.563062,246.239651
min,296.000000,0.000000,2016.000000,1.000000,1.000000,1.000000,1372.000000,0.000000
25%,758.000000,406.000000,2017.000000,4.000000,9.000000,2.000000,2281.000000,4.000000
50%,879.000000,483.000000,2018.000000,7.000000,16.000000,3.000000,2357.000000,299.000000
75%,1032.000000,545.000000,2019.000000,9.000000,23.000000,4.000000,2461.000000,452.000000
max,1459.000000,905.000000,2021.000000,12.000000,31.000000,5.000000,2921.000000,1044.000000


In [302]:
all_data['left_members_at_dinner'] = round(np.log1p(all_data['left_members_at_dinner']))
all_data = all_data.loc[(all_data['석식계'] > 0)]
all_data = all_data.loc[(all_data['left_members_at_dinner'] > 0)]
all_data.describe()

,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner
count,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000
mean,897.104082,489.146939,2018.069388,6.561224,15.780612,2.931633,2361.776531,5.167347
std,219.105176,108.567913,1.446803,3.474896,8.618567,1.520181,181.981299,1.629828
min,296.000000,104.000000,2016.000000,1.000000,1.000000,1.000000,1372.000000,1.000000
25%,746.750000,432.750000,2017.000000,3.750000,8.000000,2.000000,2274.000000,4.000000
50%,880.000000,500.000000,2018.000000,7.000000,16.000000,3.000000,2355.000000,6.000000
75%,1059.500000,557.000000,2019.000000,10.000000,23.000000,4.000000,2463.250000,6.000000
max,1459.000000,902.000000,2021.000000,12.000000,31.000000,5.000000,2921.000000,7.000000


In [303]:
#all_data.loc[all_data['석식계'] == 0, '석식계'] = 1
#all_data.loc[all_data['left_members_at_dinner'] == 0, 'left_members_at_dinner'] = 1


all_data['dinner_rates'] = all_data['석식계'] / all_data['left_members_at_dinner']
all_data['lunch_rates'] = all_data['중식계'] / all_data['left_members_at_lunch']

all_data.describe()

,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner,dinner_rates,lunch_rates
count,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000
mean,897.104082,489.146939,2018.069388,6.561224,15.780612,2.931633,2361.776531,5.167347,114.759876,0.379756
std,219.105176,108.567913,1.446803,3.474896,8.618567,1.520181,181.981299,1.629828,78.162468,0.088640
min,296.000000,104.000000,2016.000000,1.000000,1.000000,1.000000,1372.000000,1.000000,24.600000,0.143649
25%,746.750000,432.750000,2017.000000,3.750000,8.000000,2.000000,2274.000000,4.000000,81.107143,0.315576
50%,880.000000,500.000000,2018.000000,7.000000,16.000000,3.000000,2355.000000,6.000000,91.464286,0.375378
75%,1059.500000,557.000000,2019.000000,10.000000,23.000000,4.000000,2463.250000,6.000000,107.212500,0.442144
max,1459.000000,902.000000,2021.000000,12.000000,31.000000,5.000000,2921.000000,7.000000,532.000000,0.597203


In [292]:
min = all_data['lunch_rates'].min()
max = all_data['lunch_rates'].max()
all_data['lunch_rates'] = (9 * (all_data['lunch_rates'] - min) / (max - min)) + 1

min = all_data['dinner_rates'].min()
max = all_data['dinner_rates'].max()
all_data['dinner_rates'] = (9 * (all_data['dinner_rates'] - min) / (max - min)) + 1


all_data.describe()

,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner,dinner_rates,lunch_rates
count,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000
mean,897.104082,489.146939,2018.069388,6.561224,15.780612,2.931633,2361.776531,336.943878,1.431287,5.685141
std,219.105176,108.567913,1.446803,3.474896,8.618567,1.520181,181.981299,231.095375,1.353167,1.758899
min,296.000000,104.000000,2016.000000,1.000000,1.000000,1.000000,1372.000000,1.000000,1.000000,1.000000
25%,746.750000,432.750000,2017.000000,3.750000,8.000000,2.000000,2274.000000,80.750000,1.010270,4.411596
50%,880.000000,500.000000,2018.000000,7.000000,16.000000,3.000000,2355.000000,364.000000,1.016759,5.598262
75%,1059.500000,557.000000,2019.000000,10.000000,23.000000,4.000000,2463.250000,491.000000,1.083499,6.923122
max,1459.000000,902.000000,2021.000000,12.000000,31.000000,5.000000,2921.000000,1044.000000,10.000000,10.000000


In [293]:
meals = ['석식메뉴']#, '중식메뉴', '조식메뉴']
dump1 = []

for i in meals:
  for j in range(0, len(all_data)):
    tmp = all_data[i][j].split(' ')
    for k in tmp:
      if k == ' ' or k == '':
        continue
      dump1.append(k)

print(len(dump1))
mySet = list(set(dump1))
mySet2 = set()
print(len(mySet))

drop_list = ['수입산', '국내산', '중국산', '호주산', '밥', '김치']
drop_count = 0

for i in mySet:
  should_drop = False

  if i.startswith('(') and i.endswith(')'):
    should_drop = True

  for j in drop_list:
    if j in i:
     should_drop = True


  if should_drop == False:
    mySet2.add(i)

print(len(mySet2))
mySet = list(mySet)

KeyError: ignored

In [254]:
menu = [[0 for col in range(len(mySet2))] for row in range(4)]
#for i in range(0, len(mySet2)):
menu[0] = list(mySet2)
#menu[1]


In [255]:
meals = ['석식메뉴']#, '중식메뉴', '조식메뉴']

for i in meals:
  for j in range(0, len(all_data[i])):
    tmp = all_data[i][j].split(' ')
    for k in tmp:
      if k == '':
        continue

      #print(k)
      for l in range(0, len(menu[0])):
        if menu[0][l] == '':
          continue

        #print(menu[0][l]+' @ '+k+'\r\n')
        if k == menu[0][l]:
          print(menu[0][l]+'@'+k+' '+(str)(l)+'\r\n')
          menu[1][l] += all_data['dinner_rates'][j]
          menu[2][l] += 1
          print('##'+(str)(l)+' '+(str)(k)+' '+(str)(menu[1][l])+' '+(str)(menu[2][l])+'\r\n')
          break

#menu[1]

Streaming output truncated to the last 5000 lines.
고구마치즈구이@고구마치즈구이 1042

##1042 고구마치즈구이 9.154770696426038 9

홍시드레싱샐러드@홍시드레싱샐러드 191

##191 홍시드레싱샐러드 12.185627617432923 6

오이피클@오이피클 936

##936 오이피클 5.101146109516799 5

깍두기@깍두기 1276

##1276 깍두기 111.32758651105578 63

잔치국수@잔치국수 405

##405 잔치국수 37.56482103526411 31

쇠고기퀘사디아@쇠고기퀘사디아 730

##730 쇠고기퀘사디아 11.212009357060829 11

꽃맛살샐러드@꽃맛살샐러드 797

##797 꽃맛살샐러드 25.352505675117452 25

음료/단무지@음료/단무지 732

##732 음료/단무지 3.0364795998150376 3

쇠고기샤브국@쇠고기샤브국 1283

##1283 쇠고기샤브국 50.492372007199954 16

탕수어@탕수어 467

##467 탕수어 22.200185305822743 13

견과류조림@견과류조림 612

##612 견과류조림 24.89279255762039 12

양념깻잎지@양념깻잎지 1362

##1362 양념깻잎지 33.40848988265162 12

주꾸미볶음@주꾸미볶음 922

##922 주꾸미볶음 15.091675516279503 10

계란말이@계란말이 1239

##1239 계란말이 36.22820971395815 15

쫑상추무침@쫑상추무침 1245

##1245 쫑상추무침 40.09852256552827 11

유부주머니국@유부주머니국 440

##440 유부주머니국 9.160721373075006 9

통새우또띠아@통새우또띠아 1416

##1416 통새우또띠아 1.0100219730738795 1

야채쫄면무침@야채쫄면무침 875

##875 야채쫄면무침 1.0100219730738795

In [257]:
for i in range(0, len(menu[3])):
  if menu[2][i] == 0:
    print(menu[0][i]+' '+(str)(menu[1][i])+' '+(str)(menu[2][i])+' '+(str)(menu[3][i]))
  else:
    menu[3][i] = menu[1][i] / menu[2][i]

for i in range(0, len(menu[3])):
  if menu[3][i] == 0:
    print(menu[0][i]+' '+(str)(menu[1][i])+' '+(str)(menu[2][i])+' '+(str)(menu[3][i]))

In [263]:
all_data['predict_dinner_rate'] = 0

for i in range(0, len(train)):
  rates = 0
  meals = train['석식메뉴'][i]
  arr = meals.split(' ')
  for j in arr:
    for k in range(0, len(menu[0])):
      if j == menu[0][k]:
        print(j+' '+menu[0][k]+' '+(str)(menu[3][k]))
        rates += menu[3][k]
        continue

  all_data['predict_dinner_rate'][i] = rates

all_data.head(20)

육개장 육개장 2.3877571195513876
자반고등어구이 자반고등어구이 2.0930625287535634
두부조림 두부조림 1.9676005722065497
건파래무침 건파래무침 2.4464137142923175
어묵국 어묵국 2.335594741520879
유산슬 유산슬 1.0220598214366827
아삭고추무침 아삭고추무침 2.127390108327457
바나나 바나나 1.0236528608905944
청국장찌개 청국장찌개 2.840659631503781
황태양념구이 황태양념구이 3.112039784469908
고기전 고기전 1.7219912462391567
새송이버섯볶음 새송이버섯볶음 2.5466125500034797
우동 우동 1.0124579511251977
멕시칸샐러드 멕시칸샐러드 1.0154112859268238
군고구마 군고구마 1.0147518314215311
무피클 무피클 1.0126317060267933
차돌박이찌개 차돌박이찌개 2.4551441626674544
닭갈비 닭갈비 2.3544010662277794
감자소세지볶음 감자소세지볶음 1.0963039571982411
콩나물무침 콩나물무침 2.9098358273396236
맑은국 맑은국 1.0223454958919183
군만두 군만두 1.0121795975959538
과일샐러드 과일샐러드 1.0148400873479133
락교 락교 1.0119575943479895
미니함박 미니함박 1.073863536224181
어묵볶음 어묵볶음 1.9608563678993352
물파래무침 물파래무침 1.416945972689263
깍두기 깍두기 1.761041736851311
홍합미역국 홍합미역국 1.0136442221516688
임연수구이 임연수구이 1.0136442221516688
브로컬리초장 브로컬리초장 3.3288680790554044
된장찌개 된장찌개 2.7098310043836764
쇠불고기 쇠불고기 1.0131640860593414
해파리겨자채 해파리겨자채 1.0258840029

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


야채튀김 야채튀김 1.0118573806206577
조각사과 조각사과 1.0137421285951695
오복지 오복지 1.0144253420093723
오징어국 오징어국 2.4752532440604624
미니함박 미니함박 1.073863536224181
멸치호두볶음 멸치호두볶음 1.2244515453330433
치커리사과무침 치커리사과무침 1.636284551991212
쇠고기무국 쇠고기무국 1.7906909654241843
가자미찜 가자미찜 2.369831388825404
계란야채말이 계란야채말이 1.0133772593606738
돌나물오이무침 돌나물오이무침 1.0133772593606738
유부주머니국 유부주머니국 1.0178579303416673
떡볶이 떡볶이 1.0135136083362322
단무지무침 단무지무침 1.1395482623199489
오렌지 오렌지 1.4904876123141406
팽이버섯장국 팽이버섯장국 1.0169234403312317
새우또띠아 새우또띠아 1.0438621743689607
견과류조림 견과류조림 2.074399379801699
피크닉 피크닉 1.0145761179621167
들깨미역국 들깨미역국 2.409924388520446
수제돈가스 수제돈가스 1.0490019126725918
감자범벅 감자범벅 1.0255623582222273
참나물생채 참나물생채 2.524373803828382
꽃게탕 꽃게탕 2.316335125293448
유산슬 유산슬 1.0220598214366827
야채계란찜 야채계란찜 3.6471066871445026
두반장가지나물 두반장가지나물 1.707990406116818
유부장국 유부장국 1.0124124440481046
새싹피자 새싹피자 1.0113172507840642
과일샐러드 과일샐러드 1.0148400873479133
꼬들단무지 꼬들단무지 1.4681481186678607
어묵국 어묵국 2.335594741520879
닭볶음탕 닭볶음탕 2.5311650138264223
새송이버섯전 새송이버섯

0    False
1    False
2    False
3    False
4    False
Name: predict_dinner_rate, dtype: bool

In [268]:
for i in range(0, len(all_data['predict_dinner_rate'])):
  if all_data['predict_dinner_rate'][i] == 0:
    #print('@')
    print(all_data['석식메뉴'][i])

In [266]:
all_data.describe()

,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner,dinner_rates,lunch_rates,predict_dinner_rate
count,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000
mean,890.334440,461.808299,2018.052282,6.512033,15.947718,3.003320,2365.253112,274.293776,1.843476,5.619696,6.964315
std,209.505057,139.060781,1.433958,3.453906,8.650452,1.415384,176.563062,246.042929,1.629762,1.688861,3.402103
min,296.000000,1.000000,2016.000000,1.000000,1.000000,1.000000,1372.000000,1.000000,1.000000,1.000000,0.000000
25%,758.000000,406.000000,2017.000000,4.000000,9.000000,2.000000,2281.000000,4.000000,1.010430,4.454559,4.000000
50%,879.000000,483.000000,2018.000000,7.000000,16.000000,3.000000,2357.000000,299.000000,1.015580,5.529897,6.000000
75%,1032.000000,545.000000,2019.000000,9.000000,23.000000,4.000000,2461.000000,452.000000,1.140764,6.722016,9.000000
max,1459.000000,905.000000,2021.000000,12.000000,31.000000,5.000000,2921.000000,1044.000000,10.000000,10.000000,23.000000
